In [ ]:
from lerobot.robots.so101_follower import SO101FollowerConfig
from lerobot.teleoperators.so101_leader import SO101LeaderConfig

leader_port ='/dev/tty.usbmodem5A460826291'
leader_id = "my_leader"
follower_port = '/dev/tty.usbmodem5A460840631'
follower_id = "my_follower"


robot_config = SO101FollowerConfig(
    port=follower_port,
    id=follower_id,
)

teleop_config = SO101LeaderConfig(
    port=leader_port,
    id=leader_id
)

#### Calibrate the robots
```
lerobot-calibrate --robot.type=so101_follower --robot.port=/dev/tty.usbmodem5A460826291 --robot.id=my_follower
lerobot-calibrate --teleop.type=so101_leader --teleop.port=/dev/tty.usbmodem5A460840631 --teleop.id=my_leader

lerobot-teleoperate --robot.type=so101_follower --robot.port=/dev/tty.usbmodem5A460840631 --robot.id=my_follower --teleop.type=so101_leader --teleop.port=/dev/tty.usbmodem5A460826291 --teleop.id=my_leader
```

In [ ]:
from lerobot.teleoperators.so101_leader import SO101Leader

leader = SO101Leader(teleop_config)
leader.connect(calibrate=False)
leader.calibrate()
leader.disconnect()

In [ ]:
from lerobot.robots.so101_follower import SO101Follower

follower = SO101Follower(robot_config)
follower.connect()
follower.calibrate()
follower.disconnect()


To solve:
1. calibrating a geometrical drawing plane in the robots space
2. define a path of movement whereby the robots pencil moves along the path of a shape constrained to this plane

### 1) Calibrate the drawing plane

Goal: compute a rigid transform that maps 2D coordinates on your drawing plane (u, v) → 3D robot/world coordinates Pw. Also compute the plane normal so you can orient the pencil.

Inputs you’ll produce from the robot:

- a few measured 3D points (Pi) in the robot base/world frame obtained by touching the tip to known points on the drawing surface (or by probing).

**Recommended methods**
- Minimum (fast): touch three non-collinear points on the surface → compute plane exactly.
- Better (robust): probe many points (N ≥ 6) and do a least-squares plane fit (SVD).

**Math & algorithm (robust fit)**

1. Collect points Pi  ∈ R3 (robot frame).
2. Compute centroid  $\bar{p} = \frac{1}{N} \sum_i p_i$.
3. Center points: $ q_i = p_i - \bar{p}$.
4. SVD on data matrix $Q = [q_1, ..., q_N]^T$. The plane normal $ 𝑛 $ is the singular vector corresponding to smallest singular value.
5. Choose a local 2D basis on the plane: pick $ 𝑢 = normalized( p_2 - p_1 )$ (or use any vector not parallel to n), then $ v = n x n $
6. Origin of plane = $ O = \bar{p}$ (or one of the measured points).
7. The plane frame axes are $(𝑢,𝑣,𝑛)$ and origin $O$.


You now have a 4×4 transform
```
[ u.x  v.x  n.x  O.x ]
[ u.y  v.y  n.y  O.y ]
[ u.z  v.z  n.z  O.z ]
[  0    0    0    1  ]
```




#### **Projecting between frames:**

In [ ]:


def fit_plane_svd(points):
    # points: (N,3) numpy array in robot/world frame
    centroid = points.mean(axis=0)
    Q = points - centroid
    # SVD
    _, _, vh = np.linalg.svd(Q, full_matrices=False)
    normal = vh[-1, :]           # smallest singular vector
    normal = normal / np.linalg.norm(normal)
    # choose u axis: vector from first to second point projected onto plane
    tmp = points[1] - points[0]
    u = tmp - np.dot(tmp, normal) * normal
    u = u / np.linalg.norm(u)
    v = np.cross(normal, u)
    T = np.eye(4)
    T[0:3,0] = u
    T[0:3,1] = v
    T[0:3,2] = normal
    T[0:3,3] = centroid
    return T, u, v, normal, centroid

How to probe points with the robot:
- Move the robot to a position above the drawing surface.
- Lower the robot until the pencil just touches the surface (you can use force sensing or a small offset).
- Record the robot's end-effector position as a 3D point.
- Repeat for multiple points on the drawing surface.


### 2) Define and execute drawing paths constrained to that plane

Workflow : create 2D shape => sample/resample into points along path -> map points tp plane3D -> generate EE poses with correct orientation and Z offsets -> IK to joint targets -> trajectory execution with smoothing

#### Step A - Represent the 2D path

options:
- Vector format (SVG / paths) --preferred for shapes and text.
- Paramettric definitions (circle, bezier, lines).
- Raster -> contours (for image tracing).

You can parse and SVG path to a sequence of 2D curves and the sample them at resolution $ d_s $ (mm per step)

#### Step B -- Resample / smooth
- Resample so consequtive points are within a chosen distance (e.g. 0.5-2 mm).
- Apply a small smoothing filter if desired.

#### Step C -- Convert 2D sample points to 3D world
For each sampled 2D pont (s, t):
* Computer world position `p_w = O + s*u + t*v`
* Typical drawing poses:
    * `hover_pose` at `z=+h` mm above the plane
    * `contact_pose` at `z=-d` relative to the plane origin where pencil contacts (small negative offset along normal) -- or simply `z` at plane minus a small penetration if required for pressure

Make the EE orientation such that the toll frames's z-axis aligns with `-normal` (pencil pointing toward plane). You can compose a rotation matrix `R` where the third column = `-normal`.

#### Step D -- Solve IK & trajectory interpolation
* For each 3D pose (position + orientation), compute joint angles with your robots's IK.
    * If IK fails at any pose, try small perturbations, or plan in joint space.
* Option 1 -- Task-space interpolation: for eahc adjacent sample, compute IK for each sampled pose (keeps straight line in task space).
* Option 2 -- Joint-space interpolation: computer for key waypoints, then interpolate joint values (faster but may not keep pencil exactly on straight line).
* Interpolate smoothly by time using trapezoidal velocity profiles or cubic splines for position and quaternion slerp for orientations.

**Pen Up / down :**
* Implemnent `pen_up()` and `pen_down()` as :
    * Move to hover height `+h` and then move in `x,y` to next start; or
    * Use sservo to lift/lower pencil if tool supports it.
* If using pen contact via Z offset, plan gentle lowering speed and small dwell time.

**Pytho skeleton for path -> world -> IK -> execute**

```
def sample_path_2d(path2d, ds=0.001):
    # path2d: list of segments, or list of (x,y) points
    # return list of (x,y) sampled at spacing ds
    # <-- implement based on your path source (SVG, parametric)
    pass

def plane_point_to_world(T, s, t):
    # T from fit_plane_svd: columns u,v,n and origin as last column
    u = T[0:3,0]; v = T[0:3,1]; origin = T[0:3,3]
    return origin + s*u + t*v

def pose_for_point(world_pos, normal, hover=False, hover_height=0.02, contact_depth=0.0):
    # returns 4x4 pose with orientation: z axis = -normal (pencil points down)
    z = -normal
    # choose x axis along plane u (or derived)
    # Here assume we have u available; else compute arbitrary orthonormal frame
    # Build rotation R = [x y z]
    # set position = world_pos + z * (hover_height if hover else contact_depth)
    pass

# For each pose:
# 1) IK -> joints
# 2) Interpolate between joints and command robot at reasonable rate
# 3) Wait for completion or monitor joint reached

```


#### Quick example: calibrate with 3 points and draw a circle (complete small script)

```
import numpy as np
# --- assume you have functions to move robot, read EE position and solve IK for your robot ---
# probe_points = array of shape (N,3) recorded by touching surface
probe_points = np.array([
    [0.5, 0.1, 0.2],
    [0.6, 0.15, 0.2],
    [0.55, 0.2, 0.2],
])

T, u, v, normal, origin = fit_plane_svd(probe_points)

# create 2D circle path in meters, center (0,0) radius 0.05
angles = np.linspace(0, 2*np.pi, 300)
circle2d = [(0.05*np.cos(a), 0.05*np.sin(a)) for a in angles]

# convert to world poses and execute
hover_h = 0.02   # 2 cm above plane
contact_z = 0.0  # exactly on plane, or small negative for slight pressure

for i, (s,t) in enumerate(circle2d):
    pos_w = origin + s*u + t*v
    # pose: position slightly above plane for first move, then lower
    if i == 0:
        goal_pos = pos_w + (-normal)*hover_h
        move_robot_to_pose(goal_pos, orientation_from_axes(u, v, normal))
        move_robot_to_pose(pos_w + (-normal)*0.001, orientation_from_axes(u, v, normal))  # touch lightly
    else:
        goal_pos = pos_w + (-normal)*0.001
        joints = ik_solve(goal_pos, orientation_from_axes(u, v, normal))
        command_robot_joints(joints)   # with interpolation/smoothness
# lift pen
move_robot_to_pose(origin + (-normal)*hover_h, orientation_from_axes(u, v, normal))

```